In [22]:
class GridWorld:
    def __init__(self):
        self.rows = 3
        self.cols = 4
        self.num_states = self.rows * self.cols
        self.num_actions = 4
        self.actions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # Down, Left, Up, Right
        self.values = np.zeros((self.rows, self.cols))
        self.values[0, 3] = 1  # Reward of grid 3 is 1
        self.values[1, 3] = -1  # Reward of grid 7 is -1
        self.wall = (1, 1)  # Grid 5 is a wall
        self.discount_factor = 0.9
        self.noise = 0.2
        self.transition = 0.8


    def is_terminal_state(self, state):
        return state == (0, 3) or state == (1, 3)

    def get_next_state(self, state, action):
        next_row = state[0] + action[0]
        next_col = state[1] + action[1]
        # Check if next state is a wall
        if (next_row, next_col) == self.wall or next_row < 0 or next_row >= self.rows or next_col < 0 or next_col >= self.cols:
            return state
        return next_row, next_col

    def is_valid_action(self, state, action):
        next_row = state[0] + action[0]
        next_col = state[1] + action[1]

        # Check if next state is within the grid boundaries
        if next_row < 0 or next_row >= self.rows or next_col < 0 or next_col >= self.cols:
            return False

        # Check if next state is a wall
        if (next_row, next_col) == self.wall:
            return False

        return True


    def num_valid_actions(self, state):
      valid_actions = {}
      valid_actions_count = 0
      for action in self.actions:
          if self.is_valid_action(state, action):
              next_state = self.get_next_state(state, action)
              # Find the index of the action
              action_index = self.actions.index(action)
              valid_actions[action_index] = self.values[next_state]
              valid_actions_count += 1

      return valid_actions, valid_actions_count-1




    def search_state_with_value(self, start_state, visited=None):
        if visited is None:
            visited = set()
        if start_state in visited:
            return None
        visited.add(start_state)
        if self.values[start_state] == 1:
            return start_state
        for action in self.actions:
            next_state = self.get_next_state(start_state, action)
            found_state = self.search_state_with_value(next_state, visited)
            if found_state:
                return found_state


    def index_to_state(self, index):
        # Calculate row and column based on the index
        row = index // self.cols
        col = index % self.cols
        return row, col

    def find_next_state(self, state):
         for action in self.actions:
          next_state = self.get_next_state(state, action)
          if self.values[next_state] == 0:
                 return next_state



    def value_iteration(self, max_iterations=100, epsilon=0.001):
        q_values = np.zeros((12, 4))
        for iteration in range(max_iterations):
            delta = 0
            for i in range(self.rows):
                for j in range(self.cols):
                    if not self.is_terminal_state((i, j)) and (i, j) != self.wall:
                        index = i * self.cols + j
                        map, num_valid_actions = self.num_valid_actions((i, j))
                        for key, val in map.items():
                              value_of_s = val * self.transition * self.discount_factor
                              values_except = [value for k, value in map.items() if k != key]
                              for val in values_except:
                                  value_of_s += (val * self.noise * self.discount_factor) / num_valid_actions
                              q_values[index, key] = value_of_s

            max_values = [max(row) for row in q_values]
            for index in range(self.rows * self.cols):
                i, j = self.index_to_state(index)
                if not self.is_terminal_state((i, j)) and (i, j) != self.wall:
                    self.values[i, j] = max_values[index]

            self.print_grid_values(iteration+2)



    def print_grid_values(self,iteration):
        print(f"Iteration {iteration}:")
        for i in range(self.rows):
            for j in range(self.cols):
                if (i, j) == self.wall:
                    print("Wall", end=" | ")
                elif (i, j) == (0, 3):
                    print("1.0", end=" | ")
                elif (i, j) == (1, 3):
                    print("-1.0", end=" | ")
                else:
                    print(f"{self.values[i, j]:.2f}", end=" | ")
            print()


    def print_grid(self):
        print("Iteration 0")
        for i in range(self.rows):
            for j in range(self.cols):
                if (i, j) == self.wall:
                    print("Wall", end=" | ")
                else:
                   print("{:.2f}".format(0), end=" | ")
            print()


In [23]:
import numpy as np

g=GridWorld()
g.print_grid()
g.print_grid_values(1)

g.value_iteration()

Iteration 0
0.00 | 0.00 | 0.00 | 0.00 | 
0.00 | Wall | 0.00 | 0.00 | 
0.00 | 0.00 | 0.00 | 0.00 | 
Iteration 1:
0.00 | 0.00 | 0.00 | 1.0 | 
0.00 | Wall | 0.00 | -1.0 | 
0.00 | 0.00 | 0.00 | 0.00 | 
Iteration 2:
0.00 | 0.00 | 0.72 | 1.0 | 
0.00 | Wall | 0.00 | -1.0 | 
0.00 | 0.00 | 0.00 | 0.00 | 
Iteration 3:
0.00 | 0.52 | 0.72 | 1.0 | 
0.00 | Wall | 0.43 | -1.0 | 
0.00 | 0.00 | 0.00 | 0.00 | 
Iteration 4:
0.37 | 0.52 | 0.81 | 1.0 | 
0.00 | Wall | 0.43 | -1.0 | 
0.00 | 0.00 | 0.31 | 0.00 | 
Iteration 5:
0.37 | 0.65 | 0.81 | 1.0 | 
0.27 | Wall | 0.52 | -1.0 | 
0.00 | 0.22 | 0.31 | 0.04 | 
Iteration 6:
0.51 | 0.65 | 0.82 | 1.0 | 
0.27 | Wall | 0.52 | -1.0 | 
0.23 | 0.22 | 0.40 | 0.04 | 
Iteration 7:
0.51 | 0.69 | 0.82 | 1.0 | 
0.41 | Wall | 0.54 | -1.0 | 
0.23 | 0.33 | 0.40 | 0.11 | 
Iteration 8:
0.57 | 0.69 | 0.83 | 1.0 | 
0.41 | Wall | 0.54 | -1.0 | 
0.36 | 0.33 | 0.43 | 0.11 | 
Iteration 9:
0.57 | 0.70 | 0.83 | 1.0 | 
0.47 | Wall | 0.55 | -1.0 | 
0.36 | 0.37 | 0.43 | 0.13 | 
Iteration 

In [46]:
class GridWorld:
    def __init__(self):
        self.rows = 3
        self.cols = 4
        self.num_states = self.rows * self.cols
        self.num_actions = 4
        self.actions = [(1, 0), (0, -1), (-1, 0), (0, 1)]  # Down, Left, Up, Right
        self.values = np.zeros((self.rows, self.cols))
        self.values[0, 3] = 1  # Reward of grid 3 is 1
        self.values[1, 3] = -1  # Reward of grid 7 is -1
        self.wall = (1, 1)  # Grid 5 is a wall
        self.discount_factor = 0.9
        self.noise = 0.2
        self.transition = 0.8
        self.q_values = np.zeros((self.rows * self.cols, 4))


    def is_terminal_state(self, state):
        return state == (0, 3) or state == (1, 3)

    def get_next_state(self, state, action):
        next_row = state[0] + action[0]
        next_col = state[1] + action[1]
        # Check if next state is a wall
        if (next_row, next_col) == self.wall or next_row < 0 or next_row >= self.rows or next_col < 0 or next_col >= self.cols:
            return state
        return next_row, next_col

    def is_valid_action(self, state, action):
        next_row = state[0] + action[0]
        next_col = state[1] + action[1]

        # Check if next state is within the grid boundaries
        if next_row < 0 or next_row >= self.rows or next_col < 0 or next_col >= self.cols:
            return False

        # Check if next state is a wall
        if (next_row, next_col) == self.wall:
            return False

        return True


    def num_valid_actions(self, state):
      valid_actions = {}
      valid_actions_count = 0
      for action in self.actions:
          if self.is_valid_action(state, action):
              next_state = self.get_next_state(state, action)
              # Find the index of the action
              action_index = self.actions.index(action)
              valid_actions[action_index] = self.values[next_state]
              valid_actions_count += 1

      return valid_actions, valid_actions_count-1




    def search_state_with_value(self, start_state, visited=None):
        if visited is None:
            visited = set()
        if start_state in visited:
            return None
        visited.add(start_state)
        if self.values[start_state] == 1:
            return start_state
        for action in self.actions:
            next_state = self.get_next_state(start_state, action)
            found_state = self.search_state_with_value(next_state, visited)
            if found_state:
                return found_state


    def index_to_state(self, index):
        # Calculate row and column based on the index
        row = index // self.cols
        col = index % self.cols
        return row, col

    def find_next_state(self, state):
         for action in self.actions:
          next_state = self.get_next_state(state, action)
          if self.values[next_state] == 0:
                 return next_state


    def Q_values(self, max_iterations=100, epsilon=0.001):


        for iteration in range(max_iterations):
            delta = 0
            for i in range(self.rows):
                for j in range(self.cols):
                    if not self.is_terminal_state((i, j)) and (i, j) != self.wall:
                        index = i * self.cols + j
                        map, num_valid_actions = self.num_valid_actions((i, j))

                        for key, val in map.items():
                            value_of_s = val * self.transition * self.discount_factor
                            values_except = [value for k, value in map.items() if k != key]

                            for val in values_except:
                                value_of_s += (val * self.noise * self.discount_factor) / num_valid_actions

                            self.q_values[index, key] = value_of_s

            print("Iteration", iteration+2, "- Q-Values:")
            for row_index in range(0, len(self.q_values), 4):
                for row in self.q_values[row_index:row_index+4]:
                    print(" ".join("{:.3f}".format(val) for val in row))
                print()  # Add an empty line between groups of 4 rows
            print()  # Add an extra empty line between iterations

            max_values = [max(row) for row in self.q_values]
            for index in range(self.rows * self.cols):
                i, j = self.index_to_state(index)

                if not self.is_terminal_state((i, j)) and (i, j) != self.wall:
                    delta = max(delta, abs(self.values[i, j] - max_values[index]))
                    self.values[i, j] = max_values[index]

            if delta < epsilon:
                print("Converged. Stopping iterations.")
                break




    def value_iteration(self, max_iterations=100, epsilon=0.001):
        for iteration in range(max_iterations):
            delta = 0

            for i in range(self.rows):
                for j in range(self.cols):
                    if not self.is_terminal_state((i, j)) and (i, j) != self.wall:
                        index = i * self.cols + j
                        map, num_valid_actions = self.num_valid_actions((i, j))

                        for key, val in map.items():
                            value_of_s = val * self.transition * self.discount_factor
                            values_except = [value for k, value in map.items() if k != key]

                            for val in values_except:

                                value_of_s += (val* self.noise * self.discount_factor) / num_valid_actions

                            self.q_values[index, key] = value_of_s

            max_values = [max(row) for row in self.q_values]

            for index in range(self.rows * self.cols):
                i, j = self.index_to_state(index)

                if not self.is_terminal_state((i, j)) and (i, j) != self.wall:
                    delta = max(delta, abs(self.values[i, j] - max_values[index]))
                    self.values[i, j] = max_values[index]

            self.print_grid_values(iteration+2)

            if delta < epsilon:
                print("Converged. Stopping iterations.")
                break


    def print_grid_values(self,iteration):
        print(f"Iteration {iteration}:")
        for i in range(self.rows):
            for j in range(self.cols):
                if (i, j) == self.wall:
                    print("Wall", end=" | ")
                elif (i, j) == (0, 3):
                    print("1.0", end=" | ")
                elif (i, j) == (1, 3):
                    print("-1.0", end=" | ")
                else:
                    print(f"{self.values[i, j]:.2f}", end=" | ")
            print()


    def extract_policy(self):
          policy = np.empty((self.rows, self.cols), dtype=str)
          for i in range(self.rows):
              for j in range(self.cols):
                  if (i, j) == self.wall:
                      policy[i, j] = "WALL"
                  elif (i, j) == (0, 3):
                      policy[i, j] = "+1"
                  elif (i, j) == (1, 3):
                      policy[i, j] = "-1"
                  else:
                      best_action = None
                      best_value = float('-inf')
                      for idx, action in enumerate(self.actions):
                          next_state = self.get_next_state((i, j), action)

                          if value > best_value:
                              best_value = value
                              best_action = idx
                      if best_action == 0:
                          policy[i, j] = "Down"
                      elif best_action == 1:
                          policy[i, j] = "Left"
                      elif best_action == 2:
                          policy[i, j] = "Up"
                      elif best_action == 3:
                          policy[i, j] = "Right"
          self.print_policy(policy)

    def print_policy(self, policy):
        for i in range(self.rows):
            for j in range(self.cols):
                print(policy[i, j], end=" | ")
            print()
    def print_grid(self):
        print("Iteration 0")
        for i in range(self.rows):
            for j in range(self.cols):
                if (i, j) == self.wall:
                    print("Wall", end=" | ")
                else:
                   print("{:.2f}".format(0), end=" | ")
            print()


In [53]:
import numpy as np

g=GridWorld()
g.print_grid()
g.print_grid_values(1)

g.value_iteration()


Iteration 0
0.00 | 0.00 | 0.00 | 0.00 | 
0.00 | Wall | 0.00 | 0.00 | 
0.00 | 0.00 | 0.00 | 0.00 | 
Iteration 1:
0.00 | 0.00 | 0.00 | 1.0 | 
0.00 | Wall | 0.00 | -1.0 | 
0.00 | 0.00 | 0.00 | 0.00 | 
Iteration 2:
0.00 | 0.00 | 0.72 | 1.0 | 
0.00 | Wall | 0.00 | -1.0 | 
0.00 | 0.00 | 0.00 | 0.00 | 
Iteration 3:
0.00 | 0.52 | 0.72 | 1.0 | 
0.00 | Wall | 0.43 | -1.0 | 
0.00 | 0.00 | 0.00 | 0.00 | 
Iteration 4:
0.37 | 0.52 | 0.81 | 1.0 | 
0.00 | Wall | 0.43 | -1.0 | 
0.00 | 0.00 | 0.31 | 0.00 | 
Iteration 5:
0.37 | 0.65 | 0.81 | 1.0 | 
0.27 | Wall | 0.52 | -1.0 | 
0.00 | 0.22 | 0.31 | 0.04 | 
Iteration 6:
0.51 | 0.65 | 0.82 | 1.0 | 
0.27 | Wall | 0.52 | -1.0 | 
0.23 | 0.22 | 0.40 | 0.04 | 
Iteration 7:
0.51 | 0.69 | 0.82 | 1.0 | 
0.41 | Wall | 0.54 | -1.0 | 
0.23 | 0.33 | 0.40 | 0.11 | 
Iteration 8:
0.57 | 0.69 | 0.83 | 1.0 | 
0.41 | Wall | 0.54 | -1.0 | 
0.36 | 0.33 | 0.43 | 0.11 | 
Iteration 9:
0.57 | 0.70 | 0.83 | 1.0 | 
0.47 | Wall | 0.55 | -1.0 | 
0.36 | 0.37 | 0.43 | 0.13 | 
Iteration 